In [1]:
import pandas as pd
import numpy as np
import json
from rouge import Rouge
import re
import jieba
from tqdm import tqdm
import six
import logging
import os

In [2]:
def load_data(filename):
    """加载数据
    返回：[(text, summary)]
    """
    D = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            l = json.loads(l)
            text = '\n'.join([d['sentence'] for d in l['text']])
            D.append((text, l['summary']))
    return D

In [3]:
def extract_flow(inputs):
    text, summary = inputs
    texts = text_split(text, True)  # 取后maxlen句
    summaries = text_split(summary, False)
    mapping = extract_matching(texts, summaries)
    labels = sorted(set([i[1] for i in mapping]))
    pred_summary = ''.join([texts[i] for i in labels])
    metric = compute_main_metric(pred_summary, summary)
    return texts, labels, summary, metric

In [4]:
# 指标名
metric_keys = ['main', 'rouge-1', 'rouge-2', 'rouge-l']

# 计算rouge用
rouge = Rouge()

def compute_rouge(source, target, unit='word'):
    """计算rouge-1、rouge-2、rouge-l
    """
    if unit == 'word':
        source = jieba.cut(source, HMM=False)
        #HMM: 是否使用隐式马尔科夫
        target = jieba.cut(target, HMM=False)
    source, target = ' '.join(source), ' '.join(target)
    try:
        scores = rouge.get_scores(hyps=source, refs=target)
        return {
            'rouge-1': scores[0]['rouge-1']['f'],
            'rouge-2': scores[0]['rouge-2']['f'],
            'rouge-l': scores[0]['rouge-l']['f'],
        }
    except ValueError:
        return {
            'rouge-1': 0.0,
            'rouge-2': 0.0,
            'rouge-l': 0.0,
        }
    
def compute_main_metric(source, target, unit='word'):
    """计算所有metrics
    """
    metrics = compute_rouge(source, target, unit)
    metrics['main'] = (
        metrics['rouge-1'] * 0.2 + metrics['rouge-2'] * 0.4 +
        metrics['rouge-l'] * 0.4
    )
    return metrics['main']

In [5]:
def text_split(text, limited=True):
    """将长句按照标点分割为多个子句。
    """
    texts = re.split('[\n。；：，]',text)
    texts = [x for x in texts if len(x)>0]
    if limited:
        texts = texts[-256:]
    return texts

def extract_matching(texts, summaries, start_i=0, start_j=0):
    """在texts中找若干句子，使得它们连起来与summaries尽可能相似
    算法：texts和summaries都分句，然后找出summaries最长的句子，在texts
          中找与之最相似的句子作为匹配，剩下部分递归执行。
    """
    if len(texts) == 0 or len(summaries) == 0:
        return []
    i = np.argmax([len(s) for s in summaries])
    j = np.argmax([compute_main_metric(t, summaries[i], 'char') for t in texts])
    lm = extract_matching(texts[:j + 1], summaries[:i], start_i, start_j)
    rm = extract_matching(
        texts[j:], summaries[i + 1:], start_i + i + 1, start_j + j
    )
    return lm + [(start_i + i, start_j + j)] + rm

In [7]:
def convert(data):
    """分句，并转换为抽取式摘要
    """
    D = parallel_apply(
        func=extract_flow,
        iterable=tqdm(data, desc=u'转换数据'),
        workers=10,
        max_queue_size=200
    )
    total_metric = sum([d[3] for d in D])
    D = [d[:3] for d in D]
    print(u'抽取结果的平均指标: %s' % (total_metric / len(D)))
    return D

In [8]:
def parallel_apply(
    func,
    iterable,
    workers,
    max_queue_size,
    callback=None,
    dummy=False,
    random_seeds=True
):
    """多进程或多线程地将func应用到iterable的每个元素中。
    注意这个apply是异步且无序的，也就是说依次输入a,b,c，但是
    输出可能是func(c), func(a), func(b)。
    参数：
        callback: 处理单个输出的回调函数；
        dummy: False是多进程/线性，True则是多线程/线性；
        random_seeds: 每个进程的随机种子。
    """
    if dummy:
        from multiprocessing.dummy import Pool, Queue
    else:
        from multiprocessing import Pool, Queue

    in_queue, out_queue, seed_queue = Queue(max_queue_size), Queue(), Queue()
    if random_seeds is True:
        random_seeds = [None] * workers
    elif random_seeds is None or random_seeds is False:
        random_seeds = []
    for seed in random_seeds:
        seed_queue.put(seed)

    def worker_step(in_queue, out_queue):
        """单步函数包装成循环执行
        """
        if not seed_queue.empty():
            np.random.seed(seed_queue.get())
        while True:
            i, d = in_queue.get()
            r = func(d)
            out_queue.put((i, r))

    # 启动多进程/线程
    pool = Pool(workers, worker_step, (in_queue, out_queue))

    if callback is None:
        results = []

    # 后处理函数
    def process_out_queue():
        out_count = 0
        for _ in range(out_queue.qsize()):
            i, d = out_queue.get()
            out_count += 1
            if callback is None:
                results.append((i, d))
            else:
                callback(d)
        return out_count

    # 存入数据，取出结果
    in_count, out_count = 0, 0
    for i, d in enumerate(iterable):
        in_count += 1
        while True:
            try:
                in_queue.put((i, d), block=False)
                break
            except six.moves.queue.Full:
                out_count += process_out_queue()
        if in_count % max_queue_size == 0:
            out_count += process_out_queue()

    while out_count != in_count:
        out_count += process_out_queue()

    pool.terminate()

    if callback is None:
        results = sorted(results, key=lambda r: r[0])
        return [r[1] for r in results]

In [9]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

In [26]:
data_json = './sfzy_small.json'

data_random_order_json = data_json[:-5] + '_random_order.json'
data_extract_json = data_json[:-5] + '_extract.json'

data = load_data(data_json)
data = convert(data)

if os.path.exists(data_random_order_json):
    idxs = json.load(open(data_random_order_json))
else:
    idxs = list(range(len(data)))
    np.random.shuffle(idxs)
    json.dump(idxs, open(data_random_order_json, 'w'))

data = [data[i] for i in idxs]

with open(data_extract_json, 'w', encoding='utf-8') as f:
    for d in data:
        f.write(json.dumps(d, ensure_ascii=False, cls=NpEncoder) + '\n')

print(u'输入数据：%s' % data_json)
print(u'数据顺序：%s' % data_random_order_json)
print(u'输出路径：%s' % data_extract_json)

转换数据:   5%|▌         | 210/4047 [00:00<00:01, 2084.73it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Prefix dict has been built successfully.
Loading model cost 0.856 seconds.
Loading mode

抽取结果的平均指标: 0.6101703696853493
输入数据：./sfzy_small.json
数据顺序：./sfzy_small_random_order.json
输出路径：./sfzy_small_extract.json


# 转换特征向量

In [10]:
import pandas as pd
import json
import re
import os
import numpy as np
import tensorflow as tf
from sklearn import model_selection
from transformers import *
from tokenizers import BertWordPieceTokenizer, ByteLevelBPETokenizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K
from tensorflow.keras import initializers, activations
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import StratifiedKFold
from typing import Dict, List, Optional, Union
import random
from tqdm import tqdm

In [11]:
pretrained_path = '/root/zhengyanzhao/comment/emotion/model/'
config_path = os.path.join(pretrained_path, 'bert_config.json')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')
tokenizer = BertTokenizer.from_pretrained(vocab_path)
config = BertConfig.from_json_file(config_path)
config.output_hidden_states = True
bert_model = TFBertModel.from_pretrained(pretrained_path,config=config,from_pt=True)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
All PyTorch model weights were used when initializing TFBertModel.

Some weights or buffers of the PyTorch model TFBertModel were not initialized from the TF 2.0 model and are newly initialized: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def load_data(filename):
    """加载数据
    返回：[texts]
    """
    D = []
    with open(filename) as f:
        for l in f:
            texts = json.loads(l)[0]
            D.append(texts)
    return D

In [13]:
texts = load_data('./sfzy_small_extract.json')

In [14]:
average_pooling  = tf.keras.layers.GlobalAveragePooling1D()

In [6]:
def conver_token(texts):
    token_ = []
    for text in tqdm(texts,desc='转换向量'):
        token = tokenizer(text,max_length=256,truncation=True,padding=True,return_tensors="tf")
        vecotor = bert_model(token)[0]
        pooling = average_pooling(vecotor,mask=token['attention_mask'])
        token_.append(pooling)
    return token_

In [7]:
pooling =  conver_token(texts)

转换向量: 100%|██████████| 4047/4047 [1:21:20<00:00,  1.21s/it]  


In [ ]:
np.save('vector_exteact', pooling)

In [15]:
def sequence_padding(inputs, length=None, padding=0, mode='post'):
    """Numpy函数，将序列padding到同一长度
    """
    if length is None:
        length = max([len(x) for x in inputs])

    pad_width = [(0, 0) for _ in np.shape(inputs[0])]
    outputs = []
    for x in inputs:
        x = x[:length]
        if mode == 'post':
            pad_width[0] = (0, length - len(x))
        elif mode == 'pre':
            pad_width[0] = (length - len(x), 0)
        else:
            raise ValueError('"mode" argument must be "post" or "pre".')
        x = np.pad(x, pad_width, 'constant', constant_values=padding)
        outputs.append(x)
    return np.array(outputs)

In [14]:
b = sequence_padding(pooling)

In [16]:
np.save('vector_exteact', b)

# extract_model

In [16]:
class ResidualGatedConv1D(tf.keras.layers.Layer):
    """门控卷积
    """
    def __init__(self, filters, kernel_size, dilation_rate=1, **kwargs):
        super(ResidualGatedConv1D, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.dilation_rate = dilation_rate
        self.supports_masking = True

    def build(self, input_shape):
        super(ResidualGatedConv1D, self).build(input_shape)
        self.conv1d = tf.keras.layers.Conv1D(
            filters=self.filters * 2,
            kernel_size=self.kernel_size,
            dilation_rate=self.dilation_rate,
            padding='same',
        )
        self.layernorm = tf.keras.layers.LayerNormalization()

        if self.filters != input_shape[-1]:
            self.dense = tf.keras.layers.Dense(self.filters, use_bias=False)

        self.alpha = self.add_weight(
            name='alpha', shape=[1], initializer='zeros'
        )

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            inputs = inputs * mask[:, :, None]

        outputs = self.conv1d(inputs)
        # 2*filters 相当于两组filters来 一组*sigmoid(另一组)
        gate = K.sigmoid(outputs[..., self.filters:])
        outputs = outputs[..., :self.filters] * gate
        outputs = self.layernorm(outputs)

        if hasattr(self, 'dense'):
            #用于对象是否包含对应的属性值
            inputs = self.dense(inputs)

        return inputs + self.alpha * outputs

In [17]:
def bulid_extract_model(max_len,input_size,hidden_size):
    input_ = tf.keras.layers.Input((max_len,input_size))
    x = tf.keras.layers.Masking()(input_)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(hidden_size, use_bias=False)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=1)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=2)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=4)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=8)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=1)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = ResidualGatedConv1D(hidden_size, 3, dilation_rate=1)(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    out_put = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(inputs=input_, outputs=out_put)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['acc'])
    return model

In [38]:
def evaluate(model,data,data_x,threshold=0.2):
    '''
    data : [sample_num,3,…]
    0:spilt_text
    1:label
    2:summary_text
    '''
    evaluater = 0
    pred = model.predict(data_x)[:,:,0]
    # [sample_num,256]
    for d,yp in tqdm(zip(data,pred),desc='evaluating'):
        yp = yp[:len(d[0])]
        yp = np.where(yp > threshold)[0]
        pred_sum = ''.join([d[0][i] for i in yp])
        evaluater += compute_main_metric(pred_sum,d[2],'token')
    return evaluater/len(data)

In [39]:
class Evaluator(tf.keras.callbacks.Callback):
    """训练回调
    """
    def __init__(self,threshold,valid_data,valid_x,fold):
        self.best_metric = 0.0
        self.threshold = threshold
        self.valid_data = valid_data
        self.valid_x = valid_x
        self.fold = fold

    def on_epoch_end(self, epoch, logs=None):
        eva = evaluate(self.model,self.valid_data, self.valid_x, self.threshold + 0.1)
        if  eva >= self.best_metric:  # 保存最优
            self.best_metric = eva
            self.model.save_weights('weights/extract_model_%s.hdf5' % self.fold)
            print('eva raise to %s'%eva)
        else:
            print('eva is %s,not raise'%eva)

In [40]:
def data_split(data, fold, num_folds, mode):
    """划分训练集和验证集
    """
    if mode == 'train':
        D = [d for i, d in enumerate(data) if i % num_folds != fold]
    else:
        D = [d for i, d in enumerate(data) if i % num_folds == fold]

    if isinstance(data, np.ndarray):
        return np.array(D)
    else:
        return D

In [41]:
input_size = 768
hidden_size = 384
epochs = 20
batch_size = 64
threshold = 0.2
num_folds = 15
max_len = 256

In [42]:
K.clear_session()
model = bulid_extract_model(max_len,input_size,hidden_size)

In [43]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 768)]        0         
_________________________________________________________________
masking (Masking)            (None, 256, 768)          0         
_________________________________________________________________
dropout (Dropout)            (None, 256, 768)          0         
_________________________________________________________________
dense (Dense)                (None, 256, 384)          294912    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256, 384)          0         
_________________________________________________________________
residual_gated_conv1d (Resid (None, 256, 384)          886273    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256, 384)          0     

In [28]:
def load_data(filename):
    """加载数据
    返回：[(texts, labels, summary)]
    """
    D = []
    with open(filename) as f:
        for l in f:
            D.append(json.loads(l))
    return D

data = load_data('sfzy_small_extract.json')
data_x = np.load('vector_exteact.npy')
data_y = np.zeros_like(data_x[...,:1])

for i, d in enumerate(data):
    for j in d[1]:
        data_y[i][j][0] = 1

In [73]:
data_y.shape

(4047, 256, 1)

In [74]:
data_x.shape

(4047, 256, 768)

In [44]:
for fold in range(num_folds):
    train_data = data_split(data, fold, num_folds, 'train')
    valid_data = data_split(data, fold, num_folds, 'valid')
    train_x = data_split(data_x, fold, num_folds, 'train')
    valid_x = data_split(data_x, fold, num_folds, 'valid')
    train_y = data_split(data_y, fold, num_folds, 'train')
    valid_y = data_split(data_y, fold, num_folds, 'valid')
    # 启动训练
    evaluator = Evaluator(threshold,valid_data,valid_x,fold)
    model.fit(
        train_x,
        train_y,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[evaluator]
    )

Train on 3777 samples
Epoch 1/20
3776/3777 [============================>.] - ETA: 0s - loss: 0.2001 - acc: 0.8960



evaluating: 0it [00:00, ?it/s]

evaluating: 3it [00:00, 26.06it/s]

evaluating: 7it [00:00, 28.35it/s]

evaluating: 12it [00:00, 32.39it/s]

evaluating: 16it [00:00, 31.25it/s]

evaluating: 20it [00:00, 17.30it/s]

evaluating: 23it [00:01, 18.32it/s]

evaluating: 29it [00:01, 22.18it/s]

evaluating: 33it [00:01, 24.90it/s]

evaluating: 38it [00:01, 28.25it/s]

evaluating: 42it [00:01, 29.25it/s]

evaluating: 46it [00:01, 29.96it/s]

evaluating: 50it [00:01, 32.19it/s]

evaluating: 54it [00:01, 33.64it/s]

evaluating: 58it [00:02, 34.47it/s]

evaluating: 62it [00:02, 35.44it/s]

evaluating: 66it [00:02, 34.41it/s]

evaluating: 70it [00:02, 35.61it/s]

evaluating: 74it [00:02, 34.38it/s]

evaluating: 78it [00:02, 35.47it/s]

evaluating: 83it [00:02, 33.45it/s]

evaluating: 87it [00:02, 29.84it/s]

evaluating: 91it [00:03, 30.33it/s]

evaluating: 95it [00:03, 31.77it/s]

evaluating: 99it [00:03, 30.28it/s]

evaluating: 104it [00:03, 33.06it/s]

evaluating: 108it [00:03, 28.02it/s]

eval

eva raise to 0.4095275662767998
3777/3777 [==============================] - 23s 6ms/sample - loss: 0.2001 - acc: 0.8960
Epoch 2/20
3776/3777 [============================>.] - ETA: 0s - loss: 0.1489 - acc: 0.9146



evaluating: 0it [00:00, ?it/s]

evaluating: 5it [00:00, 41.73it/s]

evaluating: 11it [00:00, 45.89it/s]

evaluating: 16it [00:00, 45.34it/s]

evaluating: 20it [00:00, 32.13it/s]

evaluating: 25it [00:00, 34.68it/s]

evaluating: 29it [00:00, 33.74it/s]

evaluating: 35it [00:00, 37.58it/s]

evaluating: 39it [00:01, 37.65it/s]

evaluating: 44it [00:01, 39.17it/s]

evaluating: 50it [00:01, 43.59it/s]

evaluating: 55it [00:01, 43.57it/s]

evaluating: 62it [00:01, 48.92it/s]

evaluating: 68it [00:01, 50.07it/s]

evaluating: 74it [00:01, 51.22it/s]

evaluating: 80it [00:01, 52.98it/s]

evaluating: 86it [00:01, 49.50it/s]

evaluating: 92it [00:02, 51.55it/s]

evaluating: 99it [00:02, 52.83it/s]

evaluating: 105it [00:02, 52.18it/s]

evaluating: 113it [00:02, 56.44it/s]

evaluating: 119it [00:02, 54.71it/s]

evaluating: 127it [00:02, 58.84it/s]

evaluating: 134it [00:02, 56.78it/s]

evaluating: 141it [00:02, 56.08it/s]

evaluating: 147it [00:03, 47.72it/s]

evaluating: 153it [00:03, 46.82it/s

eva is 0.3557240250182881,not raise
3777/3777 [==============================] - 15s 4ms/sample - loss: 0.1489 - acc: 0.9146


...



evaluating: 0it [00:00, ?it/s]

evaluating: 2it [00:00, 10.58it/s]

evaluating: 5it [00:00, 11.66it/s]

evaluating: 7it [00:00, 12.90it/s]

evaluating: 9it [00:00, 14.23it/s]

evaluating: 11it [00:00, 13.77it/s]

evaluating: 13it [00:00, 14.53it/s]

evaluating: 15it [00:01, 15.10it/s]

evaluating: 17it [00:01, 14.91it/s]

evaluating: 19it [00:01, 15.36it/s]

evaluating: 21it [00:01,  8.98it/s]

evaluating: 23it [00:01,  9.92it/s]

evaluating: 25it [00:01, 11.63it/s]

evaluating: 27it [00:02, 11.75it/s]

evaluating: 29it [00:02, 11.73it/s]

evaluating: 31it [00:02, 12.64it/s]

evaluating: 33it [00:02, 13.67it/s]

evaluating: 35it [00:02, 13.95it/s]

evaluating: 37it [00:02, 13.85it/s]

evaluating: 39it [00:02, 13.97it/s]

evaluating: 41it [00:03, 14.53it/s]

evaluating: 43it [00:03, 15.26it/s]

evaluating: 45it [00:03, 14.64it/s]

evaluating: 47it [00:03, 14.08it/s]

evaluating: 49it [00:03, 14.38it/s]

evaluating: 51it [00:03, 15.08it/s]

evaluating: 54it [00:03, 17.26it/s]

evaluati

eva raise to 0.5928227877196184
3777/3777 [==============================] - 28s 8ms/sample - loss: 0.1218 - acc: 0.9267
Epoch 7/20
3776/3777 [============================>.] - ETA: 0s - loss: 0.1185 - acc: 0.9280



evaluating: 0it [00:00, ?it/s]

evaluating: 1it [00:00,  8.98it/s]

evaluating: 2it [00:00,  8.49it/s]

evaluating: 5it [00:00,  9.38it/s]

evaluating: 7it [00:00, 10.61it/s]

evaluating: 9it [00:00, 11.99it/s]

evaluating: 11it [00:00, 11.70it/s]

evaluating: 13it [00:01, 11.84it/s]

evaluating: 15it [00:01, 12.27it/s]

evaluating: 17it [00:01, 12.34it/s]

evaluating: 19it [00:01, 12.76it/s]

evaluating: 21it [00:01,  8.30it/s]

evaluating: 23it [00:02,  8.97it/s]

evaluating: 25it [00:02, 10.42it/s]

evaluating: 27it [00:02, 10.21it/s]

evaluating: 29it [00:02, 10.12it/s]

evaluating: 31it [00:02, 10.13it/s]

evaluating: 33it [00:03, 10.74it/s]

evaluating: 35it [00:03, 10.96it/s]

evaluating: 37it [00:03, 10.86it/s]

KeyboardInterrupt: 



evaluating: 37it [00:23, 10.86it/s]